# Tie-breaking votes

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
import glob
import os
import requests
from pandas import json_normalize

In [50]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.set_option("max_colwidth", None)

### Get congressional voting data for House and Senate from ProPublica

In [51]:
key = "sg5Q11oWW9pqLtmn8gVS0nOS179RousHDpeTgZkA"

In [52]:
base_url = "https://api.propublica.org/congress/v1/senate/votes/recent.json?offset="

In [53]:
links = []

for offset in range(1, 200):
    links.append(dict(chamber="senate", url=base_url + str(offset)))

In [54]:
jsons = []
for l in links:
    r = requests.get(l["url"], headers={"X-API-KEY": key})
    d = r.json()
    l["datadict"] = d
    jsons.append(l)

In [55]:
def processdict(d):
    data = json_normalize(d["datadict"]["results"], record_path=["votes"])
    return data

In [56]:
alldata = pd.concat([processdict(j) for j in jsons], sort=False).reset_index()

In [66]:
alldata_slim = alldata[
    [
        "index",
        "congress",
        "chamber",
        "session",
        "roll_call",
        "source",
        "url",
        "vote_uri",
        "question",
        "question_text",
        "description",
        "vote_type",
        "date",
        "time",
        "result",
        "tie_breaker",
        "tie_breaker_vote",
    ]
]

In [67]:
alldata_slim = alldata_slim.drop_duplicates()

In [68]:
alldata_slim[
    alldata_slim["tie_breaker"] == "Vice President of the United States"
].head()

,index,congress,chamber,session,roll_call,source,url,vote_uri,question,question_text,description,vote_type,date,time,result,tie_breaker,tie_breaker_vote
1219,19,117,Senate,1,73,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00073.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00073,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/73.json,On the Motion to Proceed,On the Motion to Proceed H.R. 1319,A bill to provide for reconciliation pursuant to title II of S. Con. Res. 5.,1/2,2021-03-04,14:35:00,Motion to Proceed Agreed to,Vice President of the United States,Yea
1238,18,117,Senate,1,73,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00073.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00073,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/73.json,On the Motion to Proceed,On the Motion to Proceed H.R. 1319,A bill to provide for reconciliation pursuant to title II of S. Con. Res. 5.,1/2,2021-03-04,14:35:00,Motion to Proceed Agreed to,Vice President of the United States,Yea
1257,17,117,Senate,1,73,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00073.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00073,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/73.json,On the Motion to Proceed,On the Motion to Proceed H.R. 1319,A bill to provide for reconciliation pursuant to title II of S. Con. Res. 5.,1/2,2021-03-04,14:35:00,Motion to Proceed Agreed to,Vice President of the United States,Yea
1276,16,117,Senate,1,73,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00073.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00073,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/73.json,On the Motion to Proceed,On the Motion to Proceed H.R. 1319,A bill to provide for reconciliation pursuant to title II of S. Con. Res. 5.,1/2,2021-03-04,14:35:00,Motion to Proceed Agreed to,Vice President of the United States,Yea
1295,15,117,Senate,1,73,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00073.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00073,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/73.json,On the Motion to Proceed,On the Motion to Proceed H.R. 1319,A bill to provide for reconciliation pursuant to title II of S. Con. Res. 5.,1/2,2021-03-04,14:35:00,Motion to Proceed Agreed to,Vice President of the United States,Yea


In [87]:
tie_breakers = (
    alldata_slim[alldata_slim["tie_breaker"] == "Vice President of the United States"]
    .groupby(["date", "description", "roll_call"])
    .agg({"index": "count"})
    .reset_index()
)

---

In [99]:
votes_url = "https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/"

In [100]:
links_votes = []

for vote in tie_breakers.roll_call.unique():
    links_votes.append(dict(chamber="senate", url=votes_url + str(vote) + ".json"))